Proof of <a class="ProveItLink" href="../../../../../../_theory_nbs_/theory.ipynb">proveit</a>.<a class="ProveItLink" href="../../../../../_theory_nbs_/theory.ipynb">physics</a>.<a class="ProveItLink" href="../../../../_theory_nbs_/theory.ipynb">quantum</a>.<a class="ProveItLink" href="../../theory.ipynb">QPE</a>.<a class="ProveItLink" href="../../theorems.ipynb#_precision_guarantee_lemma_02">_precision_guarantee_lemma_02</a> theorem
========

In [1]:
import proveit
theory = proveit.Theory() # the theorem's theory
from proveit import a, b, m, x, N, defaults
from proveit.logic import InSet
from proveit.numbers import two, Div, Floor, LessEq, ModAbs, Mult, Real, subtract
from proveit.numbers.modular import mod_abs_of_difference_bound, mod_abs_scaled
from proveit.numbers.rounding import floor_x_le_x
from proveit.physics.quantum.QPE import (
    _best_floor_def, _n_in_natural_pos, _phase,
    _phase_in_interval, _t_in_natural_pos, _two_pow_t,
    _two_pow_t__minus_one, _two_pow_t_minus_one_is_nat_pos)

In [2]:
%proving _precision_guarantee_lemma_02

In [3]:
defaults.assumptions = _precision_guarantee_lemma_02.conditions

In [4]:
# for convenience
_thm_expr = _precision_guarantee_lemma_02.instance_expr

Import some basic local assumptions and local definitions

In [5]:
# some basic assumptions and definitions
display(_t_in_natural_pos)
display(_n_in_natural_pos)
display(_phase_in_interval)
display(_best_floor_def)

|- t in NaturalPos

|- n in NaturalPos

|- phase in [0,1)

|- b_{f} = floor(2^{t} * phase)

It is useful generally to establish that $2^{t}\varphi \ge \lfloor 2^{t}\varphi\rfloor$ (this is then used implicitly in some auto_simplification further below) (might be nice to convert this to a method of some sort in the Floor class?):

In [6]:
floor_x_le_x

|- forall_{x in Real} (floor(x) <= x)

In [7]:
_x_sub = Mult(_two_pow_t, _phase)
floor_int_phase_bound = floor_x_le_x.instantiate({x: _x_sub}).reversed()

We gradually transform the assumed condition $|m-b|_{\text{mod}\;2^t} \le 2^{t-1}-1$ into the desired condition $|\frac{m}{2^{t}}-1|_{\text{mod}\;1} \le 2^{-n}$:

In [8]:
condition_01 = _precision_guarantee_lemma_02.condition.operands[1].prove()

In [9]:
condition_02 = (
    condition_01.inner_expr().lhs.value.operands[1].
    operand.substitute(_best_floor_def))

In [10]:
mod_abs_of_difference_bound

|- forall_{a in Real, b in Real, N in RealPos} (|a - b|_{mod N} >= (|a|_{mod N} - |b|_{mod N}))

Then for later auto-simplification to work well, we want to establish that $|\lfloor 2^{t}\varphi\rfloor - 2^{t}\varphi| \le 2^{t}/2$

In [11]:
div_mult_id = Div(_two_pow_t, two).cancelation(two)

The `_two_pow_t_minus_one_is_nat_pos` theorem, imported at the top, also lets us know that $\vdash 2^{t-1}\in\mathbb{N}$.

In [12]:
from proveit.numbers.rounding import real_minus_floor_interval
real_minus_floor_interval

|- forall_{x in Real} ((x - floor(x)) in [0,1))

In [13]:
from proveit import x
from proveit.numbers import two, Exp
from proveit.physics.quantum.QPE import _t
_x_sub = Mult(Exp(two, _t), _phase)
real_minus_floor = real_minus_floor_interval.instantiate({x: _x_sub})

In [14]:
from proveit.numbers import zero, one, Abs, IntervalCO
real_minus_floor_abs = InSet(Abs(real_minus_floor.element), IntervalCO(zero, one)).prove()

In [15]:
real_minus_floor_abs.inner_expr().element.reverse_difference(auto_simplify=False)

|- |floor(2^{t} * phase) - (2^{t} * phase)| in [0,1)

Notice in the result of the following instantiation the ModAbs that would have appeared on the lhs with respect to $2^{t}$ has been automatically simplified to a non-modular (and non-absolute-value) expression:

In [16]:
_a_sub, _b_sub, _N_sub = (
    subtract(m, Mult(_two_pow_t, _phase)),
    subtract(m, Floor(Mult(_two_pow_t, _phase))),
    _two_pow_t)
condition_03 = mod_abs_of_difference_bound.instantiate(
        {a: _a_sub, b: _b_sub, N: _N_sub}).inner_expr().lhs.dividend.commute(0,1)

In [17]:
condition_04 = condition_03.inner_expr().lhs.commute(0, 1).with_styles(direction='normal')

But we also know from our work above that $2^{t}\varphi - \lfloor 2^{t}\varphi \rfloor < 1$, so Prove-It can now give us the following:

In [18]:
condition_05 = LessEq(condition_04.lhs, one).prove()

And then we rearrange a bit:

In [19]:
condition_06 = condition_05.left_add_both_sides(condition_05.lhs.operands[1].operand)

From an earlier condition, we can derive a bound on the rhs of `condition_06` above:

In [20]:
condition_07 = condition_02.right_add_both_sides(one)

Then apply transitivity to obtain the following:

In [21]:
condition_08 = condition_06.apply_transitivity(condition_07)

We want to transform the $\text{mod}\;2^{t}$ on the lhs to the $\text{mod}\;1$ that we want in the eventual desired expression.<br/>
We can utilize the following theorem and subsequent instantiation to do that:

In [22]:
mod_abs_scaled

|- forall_{a, b, c in Real} ((a * |b|_{mod c}) = |a * b|_{mod a * c})

In [23]:
from proveit import a, b, c
_a_sub, _b_sub, _c_sub = _two_pow_t, _thm_expr.lhs.operands[0], one
condition_equality_01 = mod_abs_scaled.instantiate({a: _a_sub, b: _b_sub, c: _c_sub})

In [24]:
# clean up rhs side by distributing that factor of 2^t
condition_equality_02 = condition_equality_01.inner_expr().rhs.operands[0].distribute(1)

Now we utilize that result, substituting the lhs for the rhs in our previous result:

In [25]:
condition_09 = condition_equality_02.sub_left_side_into(condition_08)

We can then divide both sides by the factor $2^{t}$ and simplify, and we are done:

In [26]:
condition_10 = condition_09.divide_both_sides(_two_pow_t)

_precision_guarantee_lemma_02 may now be readily provable (assuming required theorems are usable).  Simply execute "%qed".


In [27]:
%qed

proveit.physics.quantum.QPE._precision_guarantee_lemma_02 has been proven.


step type	requirements	statement
0	generalization	1	|- forall_{m in {0 .. 2^{t} - 1} | |m - b_{f}|_{mod 2^{t}} <= (2^{t - n} - 1)} (|(m / 2^{t}) - phase|_{mod 1} <= 2^{-n})
1	instantiation	163, 2, 3	{m in {0 .. 2^{t} - 1}, |m - b_{f}|_{mod 2^{t}} <= (2^{t - n} - 1)} |- |(m / 2^{t}) - phase|_{mod 1} <= 2^{-n}
	P(_a) : |(m / 2^{t}) - phase|_{mod 1} <= _a, x : 2^{t - n} / 2^{t}, y : 2^{-n}
2	instantiation	4, 255, 5, 99, 6, 7, 8*	{m in {0 .. 2^{t} - 1}, |m - b_{f}|_{mod 2^{t}} <= (2^{t - n} - 1)} |- |(m / 2^{t}) - phase|_{mod 1} <= (2^{t - n} / 2^{t})
	a : 2^{t}, x : 2^{t} * |(m / 2^{t}) - phase|_{mod 1}, y : 2^{t - n}
3	instantiation	9, 77, 173, 133, 10*	|- (2^{t - n} / 2^{t}) = 2^{-n}
	x : 2^{t - n}, y : 2^{t}
4	conjecture		|- forall_{a, x, y in Real | x <= y, a > 0} ((x / a) <= (y / a))
	proveit.numbers.division.weak_div_from_numer_bound__pos_denom
5	instantiation	254, 255, 40	{m in {0 .. 2^{t} - 1}} |- (2^{t} * |(m / 2^{t}) - phase|_{mod 1}) in Real
	a : 2^{t}, b : |(m / 2^{t}) - phase|_{mod 1}
6	instantiation	177, 11, 12	{m in {0 .. 2^{t} - 1}, |m - b_{f}|_{mod 2^{t}} <= (2^{t - n} - 1)} |- (2^{t} * |(m / 2^{t}) - phase|_{mod 1}) <= 2^{t - n}
	P(_a) : _a <= 2^{t - n}, x : 2^{t} * |(m / 2^{t}) - phase|_{mod 1}, y : |m - (2^{t} * phase)|_{mod 2^{t}}
7	instantiation	13, 137	|- 2^{t} > 0
	x : 2^{t}
8	instantiation	226, 14, 15	{m in {0 .. 2^{t} - 1}} |- ((2^{t} * |(m / 2^{t}) - phase|_{mod 1}) / 2^{t}) = |(m / 2^{t}) - phase|_{mod 1}
	x : (2^{t} * |(m / 2^{t}) - phase|_{mod 1}) / 2^{t}, y : |(m / 2^{t}) - phase|_{mod 1} / 1, z : |(m / 2^{t}) - phase|_{mod 1}
9	conjecture		|- forall_{x, y in Complex | y != 0} ((x / y) = (x * y^{-1}))
	proveit.numbers.division.div_as_mult
10	instantiation	226, 16, 17	|- (2^{t - n} * (2^{t})^{-1}) = 2^{-n}
	x : 2^{t - n} * (2^{t})^{-1}, y : 2^{t - n} * 2^{-t}, z : 2^{-n}
11	instantiation	46, 18, 19	{m in {0 .. 2^{t} - 1}, |m - b_{f}|_{mod 2^{t}} <= (2^{t - n} - 1)} |- |m - (2^{t} * phase)|_{mod 2^{t}} <= 2^{t - n}
	x : |m - (2^{t} * phase)|_{mod 2^{t}}, y : |m - floor(2^{t} * phase)|_{mod 2^{t}} + 1, z : 2^{t - n}
12	instantiation	163, 20, 21	{m in {0 .. 2^{t} - 1}} |- (2^{t} * |(m / 2^{t}) - phase|_{mod 1}) = |m - (2^{t} * phase)|_{mod 2^{t}}
	P(_a) : (2^{t} * |(m / 2^{t}) - phase|_{mod 1}) = |_a|_{mod 2^{t}}, x : 2^{t} * ((m / 2^{t}) - phase), y : m - (2^{t} * phase)
13	conjecture		|- forall_{x in RealPos} (x > 0)
	proveit.numbers.number_sets.real_numbers.positive_if_in_real_pos
14	instantiation	184, 101, 186, 22, 140*	{m in {0 .. 2^{t} - 1}} |- ((2^{t} * |(m / 2^{t}) - phase|_{mod 1}) / 2^{t}) = (|(m / 2^{t}) - phase|_{mod 1} / 1)
	x : 2^{t}, z : 1, y : |(m / 2^{t}) - phase|_{mod 1}
15	instantiation	203, 22	{m in {0 .. 2^{t} - 1}} |- (|(m / 2^{t}) - phase|_{mod 1} / 1) = |(m / 2^{t}) - phase|_{mod 1}
	x : |(m / 2^{t}) - phase|_{mod 1}
16	instantiation	182, 23	|- (2^{t - n} * (2^{t})^{-1}) = (2^{t - n} * 2^{-t})
	f(_a) : 2^{t - n} * _a, x : (2^{t})^{-1}, y : 2^{-t}
17	instantiation	24, 217, 25, 86, 42, 26*	|- (2^{t - n} * 2^{-t}) = 2^{-n}
	a : 2, b : t - n, c : -t
18	instantiation	27, 115, 28, 266, 29, 30*	{m in {0 .. 2^{t} - 1}} |- |m - (2^{t} * phase)|_{mod 2^{t}} <= (|m - floor(2^{t} * phase)|_{mod 2^{t}} + 1)
	a : |m - floor(2^{t} * phase)|_{mod 2^{t}}, x : |m - (2^{t} * phase)|_{mod 2^{t}} - |m - floor(2^{t} * phase)|_{mod 2^{t}}, y : 1
19	instantiation	31, 266, 115, 32, 33, 34*	{m in {0 .. 2^{t} - 1}, |m - b_{f}|_{mod 2^{t}} <= (2^{t - n} - 1)} |- (|m - floor(2^{t} * phase)|_{mod 2^{t}} + 1) <= 2^{t - n}
	a : 1, x : |m - floor(2^{t} * phase)|_{mod 2^{t}}, y : 2^{t - n} - 1
20	instantiation	35, 255, 57, 266, 140*	{m in {0 .. 2^{t} - 1}} |- (2^{t} * |(m / 2^{t}) - phase|_{mod 1}) = |2^{t} * ((m / 2^{t}) - phase)|_{mod 2^{t}}
	a : 2^{t}, b : (m / 2^{t}) - phase, c : 1
21	instantiation	36, 288, 239, 241, 173, 37, 38, 39*	{m in {0 .. 2^{t} - 1}} |- (2^{t} * ((m / 2^{t}) - phase)) =  \\ (m - (2^{t} * phase))
	i : 1, j : 0, a : (2^{t}), x : m / 2^{t}, y : phase, b : ()
22	instantiation	286, 259, 40	